In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from my_modules import risk
from my_modules import data
from my_modules import portfolio
from my_modules import index
from my_modules import model

import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

## Week 3 Quiz

In [3]:
import matplotlib.pyplot as plt
def show_cppi(n_scenarios=50, mu=0.07, sigma=0.15, m=3, floor=0., riskfree_rate=0.03, y_max=100):
    """
    Plot the results of a Monte Carlo Simulation of CPPI
    """
    start = 100
    sim_rets = model.gbm(n_scenarios=n_scenarios, mu=mu, sigma=sigma, prices=False, steps_per_year=12)
    risky_r = pd.DataFrame(sim_rets)
    # run the "back"-test
    btr = portfolio.cppi(risky_r=pd.DataFrame(risky_r),riskfree_rate=riskfree_rate,m=m, start_value=start, floor=floor)
    wealth = btr["Wealth"]
    y_max=wealth.values.max()*y_max/100
    terminal_wealth = wealth.iloc[-1]
    
    tw_mean = terminal_wealth.mean()
    tw_median = terminal_wealth.median()
    failure_mask = np.less(terminal_wealth, start*floor)
    n_failures = failure_mask.sum()
    p_fail = n_failures/n_scenarios
    
    # When there is a failure, what is the average failure extent
    e_shortfall = np.dot(terminal_wealth-start*floor, failure_mask)/n_failures if n_failures > 0 else 0.0
    #Plot
    fig,(wealth_ax, hist_ax) = plt.subplots(nrows=1, ncols=2, sharey=True, gridspec_kw={'width_ratios':[3,2]}, figsize=(24,9))
    plt.subplots_adjust(wspace=0.0)
    
    wealth.plot(ax=wealth_ax, legend=False, alpha=0.3, color='indianred')
    wealth_ax.axhline(y=start, ls=':',color='black')
    wealth_ax.axhline()
    wealth_ax.set_ylim(top=y_max)
    
    terminal_wealth.plot.hist(ax=hist_ax, bins=50,ec='w',fc='indianred', orientation='horizontal')
    hist_ax.axhline(y=start, ls=':',color='black')
    hist_ax.annotate(f'Mean: ${int(tw_mean)}', xy=(.7,.9), xycoords='axes fraction',fontsize=24)
    hist_ax.annotate(f'Median: ${int(tw_median)}', xy=(.7,.85), xycoords='axes fraction',fontsize=24)
    if(floor > 0.01):
        hist_ax.axhline(y=start*floor, ls='--',color='red', linewidth=3)
        hist_ax.annotate(f'Violations: {n_failures} ({p_fail*100:2.2f}%)\nE(Shortfall)=${e_shortfall:2.2f}', xy=(.7,.7), xycoords='axes fraction', fontsize=24)

cppi_controls = widgets.interactive(show_cppi, 
                                   n_scenarios=widgets.IntSlider(min=1, max=1000, step=5, value=50), 
                                   mu=(0., +.2, .01),
                                   sigma=(0, .40, .05),
                                   floor=(0, 2, .1),
                                   m=(1, 5, .5),
                                   riskfree_rate=(0, .05, .01),
                                   y_max=widgets.IntSlider(min=0, max=100, step=1, value=100,
                                                           description="Zoom Y Axis")
)
display(cppi_controls)

interactive(children=(IntSlider(value=50, description='n_scenarios', max=1000, min=1, step=5), FloatSlider(val…

## _The questions didn't need to be coded, it was intuitive questions asking about changes in violations and shortfall should different paramters change or rebalancing done more often_